# Install, Paths and Parameters

In [1]:
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from tqdm import tqdm
import random
import sys

from torch.utils.tensorboard import SummaryWriter

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
# from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import *
from src.model.train import *
from src.model.multihead_model import *

from torchattacks import *
from sklearn import preprocessing

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')

LOG_BASE_PATH = Path(MAX_PATH, 'logs')

# DamageNet
DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')

# Image Net
ORI_PATH = Path(DATA_PATH, 'ori_data')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

VAL_PATH = Path(ORI_PATH, 'validation')
VAL_IMAGES_PATH = Path(VAL_PATH,'images')
VAL_LABEL_PATH = Path(VAL_PATH, 'correct_labels.txt')

TRAIN_PATH = Path(ORI_PATH, 'train')
TRAIN_IMAGES_PATH = Path(TRAIN_PATH,'images')
TRAIN_LABEL_PATH = Path(TRAIN_PATH, 'correct_labels.txt')

# Adversarial Data
# PGD
PGD_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'pgd_06', 'train')
PGD_TRAIN_IMAGES_PATH = Path(PGD_TRAIN_PATH,'images')
PGD_TRAIN_LABEL_PATH = Path(PGD_TRAIN_PATH, 'labels.txt')

PGD_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'pgd_06', 'validation')
PGD_VAL_IMAGES_PATH = Path(PGD_VAL_PATH,'images')
PGD_VAL_LABEL_PATH = Path(PGD_VAL_PATH, 'labels.txt')

# CW
CW_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'cw', 'train')
CW_TRAIN_IMAGES_PATH = Path(CW_TRAIN_PATH,'images')
CW_TRAIN_LABEL_PATH = Path(CW_TRAIN_PATH, 'labels.txt')

CW_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'cw', 'validation')
CW_VAL_IMAGES_PATH = Path(CW_VAL_PATH,'images')
CW_VAL_LABEL_PATH = Path(CW_VAL_PATH, 'labels.txt')

# FGSM
FGSM_TRAIN_PATH = Path(MAX_PATH, 'adversarial_data', 'fgsm_06', 'train')
FGSM_TRAIN_IMAGES_PATH = Path(FGSM_TRAIN_PATH,'images')
FGSM_TRAIN_LABEL_PATH = Path(FGSM_TRAIN_PATH, 'labels.txt')

FGSM_VAL_PATH = Path(MAX_PATH, 'adversarial_data', 'fgsm_06', 'validation')
FGSM_VAL_IMAGES_PATH = Path(FGSM_VAL_PATH,'images')
FGSM_VAL_LABEL_PATH = Path(FGSM_VAL_PATH, 'labels.txt')


# TB LOG
TB_LOGS_BASE_PATH = Path(LOG_BASE_PATH, 'tb_logs')


# Model save path
ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH = Path(MAX_PATH, 'adversarial_data', 'adv_classifiers')

In [2]:
ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH

PosixPath('/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers')

In [3]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
# INDEX_SUBSET = get_random_indexes(number_of_images = 50000, n_samples=1000)
# CLASS_SUBSET = get_random_classes(number_of_classes = 25, min_rand_class = 1, max_rand_class = 1001)


CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)

# CLASS_SUBSET = CLASS_SUBSET[:25] 

NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 16

DEVICE = 'cuda'

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [5]:
model, linear_classifier = get_dino()

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


# Load data

In [6]:
# Remember to set the correct transformation
# encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit([i for i in CLASS_SUBSET])


# PGD
pgd_train_dataset = AdvTrainingImageDataset(PGD_TRAIN_IMAGES_PATH, 
                                            PGD_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                            label_encoder=label_encoder
                                           )

pgd_val_dataset = AdvTrainingImageDataset(PGD_VAL_IMAGES_PATH, 
                                          PGD_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                          label_encoder=label_encoder
                                         )


pgd_train_loader = DataLoader(pgd_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

pgd_val_loader = DataLoader(pgd_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# CW
cw_train_dataset = AdvTrainingImageDataset(CW_TRAIN_IMAGES_PATH, 
                                            CW_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                           label_encoder=label_encoder
                                          )

cw_val_dataset = AdvTrainingImageDataset(CW_VAL_IMAGES_PATH, 
                                          CW_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                         label_encoder=label_encoder
                                        )

cw_train_loader = DataLoader(cw_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

cw_val_loader = DataLoader(cw_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# FGSM
fgsm_train_dataset = AdvTrainingImageDataset(FGSM_TRAIN_IMAGES_PATH, 
                                            FGSM_TRAIN_LABEL_PATH, 
                                            ORIGINAL_TRANSFORM, 
                                            CLASS_SUBSET, 
                                            index_subset=None,
                                            label_encoder=label_encoder
                                            )

fgsm_val_dataset = AdvTrainingImageDataset(FGSM_VAL_IMAGES_PATH, 
                                          FGSM_VAL_LABEL_PATH, 
                                          ORIGINAL_TRANSFORM, 
                                          CLASS_SUBSET, 
                                          index_subset=None,
                                          label_encoder=label_encoder
                                          )

fgsm_train_loader = DataLoader(fgsm_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

fgsm_val_loader = DataLoader(fgsm_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY, 
                            shuffle=False)

# Clean
clean_train_dataset = ImageDataset(TRAIN_IMAGES_PATH, 
                                  TRAIN_LABEL_PATH, 
                                  ORIGINAL_TRANSFORM,
                                  CLASS_SUBSET, 
                                  index_subset=None, 
                                  label_encoder=label_encoder)

clean_val_dataset = ImageDataset(VAL_IMAGES_PATH, 
                                  VAL_LABEL_PATH, 
                                  ORIGINAL_TRANSFORM,
                                  CLASS_SUBSET, 
                                  index_subset=None, 
                                  label_encoder=label_encoder)

clean_train_loader = DataLoader(clean_train_dataset, 
                              batch_size=BATCH_SIZE, 
                              num_workers=NUM_WORKERS, 
                              pin_memory=PIN_MEMORY, 
                              shuffle=False)

clean_val_loader = DataLoader(clean_val_dataset, 
                            batch_size=BATCH_SIZE, 
                            num_workers=NUM_WORKERS, 
                            pin_memory=PIN_MEMORY,
                            shuffle=False)


loader_dict = {
    "pgd" : {
        "train" : pgd_train_loader,
        "validation" : pgd_val_loader,
    },
    "cw" : {
        "train" : cw_train_loader,
        "validation" : cw_val_loader,
    }, 
    "fgsm" : {
        "train" : fgsm_train_loader,
        "validation" : fgsm_val_loader,
    },
    "clean" : {
        "train" : clean_train_loader,
        "validation" : clean_val_loader,
    },
}




In [7]:
loader_dict["pgd"]["train"]

In [8]:
version = '25_classes_full_v1'

## Classifier

In [9]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)


In [10]:
# Linear Binary Classifier
class LinearBC(nn.Module):
    def __init__(self, input_shape):
        super(LinearBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

## Train various classifiers on all adversarial datasets

In [23]:
for attack, loaders in loader_dict.items():
    
    # Initialise classifier
    adv_linear_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                         num_labels=len(CLASS_SUBSET))
    adv_linear_classifier = adv_linear_classifier.cuda()

    # Metric logger path
    LOG_PATH = Path(LOG_BASE_PATH, 'adv_classifier', version, attack)
    if not os.path.isdir(LOG_PATH):
        os.makedirs(LOG_PATH)
    
    # train
    pstr = "#"*50 + f"Training classifier for {attack}"+ "#"*50
    print(len(pstr)*"#")
    print(pstr)
    print(len(pstr)*"#")
    loggers = train(model, 
                    adv_linear_classifier, 
                    loaders["train"], 
                    loaders["validation"], 
                    log_dir=LOG_PATH, 
                    tensor_dir=None, 
                    optimizer=None, 
                    adversarial_attack=None,
                    criterion=nn.CrossEntropyLoss(),
                    epochs=5, 
                    val_freq=1, 
                    batch_size=16,  
                    lr=0.001, 
                    to_restore = {"epoch": 0, "best_acc": 0.}, 
                    n=4, 
                    avgpool_patchtokens=False, 
                    show_image=False)
    
    # Save adversarial Classifier
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    save_file_model = f"{attack}.pt"
    save_file_log = f"log_{attack}.pt"
    torch.save(adv_linear_classifier.state_dict(), str(save_path) + "/" + save_file_model)
    torch.save(loggers, str(save_path) + "/" + save_file_log)
    print(f'Finished Training, saving model to {str(save_path)}/{save_file_model} and log to {str(save_path)}/{save_file_model}')

###############################################################################################################################
##################################################Training classifier for pgd##################################################
###############################################################################################################################
Found checkpoint at /cluster/scratch/mmathys/dl_data/logs/adv_classifier/25_classes_full_v1/pgd/checkpoint.pth.tar
=> loaded 'state_dict' from checkpoint '/cluster/scratch/mmathys/dl_data/logs/adv_classifier/25_classes_full_v1/pgd/checkpoint.pth.tar' with msg <All keys matched successfully>
=> loaded 'optimizer' from checkpoint: '/cluster/scratch/mmathys/dl_data/logs/adv_classifier/25_classes_full_v1/pgd/checkpoint.pth.tar'
=> loaded 'scheduler' from checkpoint: '/cluster/scratch/mmathys/dl_data/logs/adv_classifier/25_classes_full_v1/pgd/checkpoint.pth.tar'
Epoch: [1]  [ 0/82]  eta: 0:00:30  lr: 0.000000  los

## Evaluation

### Evaluate on all adversarial datasets

In [1]:
attacks = [x for x in loader_dict.keys()]

for attack in attacks:
    pstr = "#"*30 + f''' evaluating adv_classifier trained on {attack} ''' + "#"*30
    print(len(pstr)*"#")
    print(pstr)
    print(len(pstr)*"#")
    adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                             num_labels=len(CLASS_SUBSET))
    adv_classifier.to(device)
    
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    save_file = f"{attack}.pt"
    adv_classifier.load_state_dict(torch.load(str(save_path) + "/" + save_file))
    
    for applied_attack in attacks:
        
#         if applied_attack == attack:
#             continue
        
        print("-"*50 + f" {applied_attack} dataset " + "-"*50)
        logger_dict, logger = validate_network(model, 
                                               adv_classifier, 
                                               loader_dict[applied_attack]["validation"], 
                                               criterion=nn.CrossEntropyLoss(),
                                               tensor_dir=None, 
                                               adversarial_attack=None, 
                                               n=4, 
                                               avgpool_patchtokens=False, 
                                               path_predictions=None)


NameError: name 'loader_dict' is not defined

### Evaluate on newly generated attacks

In [25]:
attacks = [x for x in loader_dict.keys()]
for attack in attacks:
    
    pstr = "#"*30 + f''' evaluating adv_classifier trained on {attack} ''' + "#"*30
    print(len(pstr)*"#")
    print(pstr)
    print(len(pstr)*"#")
    
    adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                             num_labels=len(CLASS_SUBSET))
    adv_classifier.to(device)
    
    save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version)
    save_file = f"{attack}.pt"
    adv_classifier.load_state_dict(torch.load(str(save_path) + "/" + save_file))
    
    vits = ViTWrapper(model, adv_classifier, transform=None)

    for applied_attack in attacks:
        
        if applied_attack == "pgd":
            ev_attack = PGD(vits, eps=0.3, alpha=6/255, steps=15)
        elif applied_attack == "cw":
            ev_attack = CW(vits, c=10, lr=0.003, steps=30)
        elif applied_attack == "fgsm":
            ev_attack = FGSM(vits, eps=0.03)
        else:
            continue
        
        print("-"*50 + f''' applying attack: {ev_attack} ''' + "-"*50)
        logger_dict, logger = validate_network(model, 
                                               adv_classifier, 
                                               loader_dict["clean"]["validation"],
                                               criterion=nn.CrossEntropyLoss(),
                                               tensor_dir=None, 
                                               adversarial_attack=ev_attack,
                                               n=4, 
                                               avgpool_patchtokens=False, 
                                               path_predictions=None)

##############################################################################################################################################
################################################## evaluating adv_classifier trained on pgd ##################################################
##############################################################################################################################################
-------------------------------------------------- applying attack: PGD(model_name=ViTWrapper, device=cuda:0, eps=0.3, alpha=0.023529411764705882, steps=15, random_start=True, attack_mode=default, return_type=float) --------------------------------------------------
Test:  [ 0/79]  eta: 0:02:02  loss: 11.795971 (11.795971)  acc1: 12.500000 (12.500000)  acc5: 25.000000 (25.000000)  adv_loss: 44.400982 (44.400982)  adv_acc1: 0.000000 (0.000000)  adv_acc5: 0.000000 (0.000000)  time: 1.550437  data: 0.273152  max mem: 1433
Test:  [20/79]  eta: 0:01:26  loss: 11.416915 

### Evaluate on full pipeline with post-hoc as multiplexer

In [11]:
# Load clean_classifier
name="clean"
clean_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                    num_labels=len(CLASS_SUBSET))
clean_classifier.to(device)
clean_classifier.load_state_dict(torch.load(f"/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/{version}/{name}.pt"))

<All keys matched successfully>

In [13]:
# Load posthoc
posthocs=["cw", "fgsm_06", "pgd_06"]

adv_models = ["cw", "fgsm", "pgd"]

# Perform validation on clean dataset
for post_model in posthocs:
    posthoc = LinearBC(1536)
    posthoc.to(device)
    posthoc.load_state_dict(torch.load(f'''/cluster/scratch/mmathys/dl_data/posthoc-models/{post_model}.pt'''))
    
    for adv_model in adv_models:
        adv_classifier = LinearClassifier(linear_classifier.linear.in_features, 
                                          num_labels=len(CLASS_SUBSET))
        adv_classifier.to(device)
        adv_classifier.load_state_dict(torch.load(f'''/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/{version}/{adv_model}.pt'''))
        
        for attack, loaders in loader_dict.items():
            print("#"*30 + f''' Validating Posthoc: {post_model} and adv_classifier: {adv_model} on {attack} ''' + "#"*30)
            log_dict, logger = validate_multihead_network(model, 
                                                          posthoc,
                                                          adv_classifier,
                                                          clean_classifier,
                                                          loader_dict[attack]["validation"], 
                                                          tensor_dir=None, 
                                                          adversarial_attack=None, 
                                                          n=4, 
                                                          avgpool=False)
            
            # Save adversarial Classifier
            save_path = Path(ADVERSARIAL_CLASSIFIER_MODEL_SAVE_PATH, version, "benchmark")
            if not os.path.isdir(save_path):
                os.makedirs(save_path)
            save_file_log = f"log_post{post_model}_adv{adv_model}_attack{attack}.pt"
            torch.save(logger, str(save_path) + "/" + save_file_log)
            

################################################## Validating Posthoc: cw and adv_classifier: cw on pgd ##################################################
Test:  [0/4]  eta: 0:00:00  loss: 1.307345 (1.307345)  acc1: 75.000000 (75.000000)  acc5: 87.500000 (87.500000)  time: 0.214432  data: 0.173989  max mem: 210
Test:  [3/4]  eta: 0:00:00  loss: 1.143139 (1.133734)  acc1: 75.000000 (78.000000)  acc5: 87.500000 (86.000000)  time: 0.219044  data: 0.185667  max mem: 210
Test: Total time: 0:00:00 (0.219371 s / it)
* Acc@1 78.000 Acc@5 86.000 loss 1.134
################################################## Validating Posthoc: cw and adv_classifier: cw on cw ##################################################
Test:  [0/4]  eta: 0:00:02  loss: 5.649935 (5.649935)  acc1: 50.000000 (50.000000)  acc5: 62.500000 (62.500000)  time: 0.534863  data: 0.494440  max mem: 210
Test:  [3/4]  eta: 0:00:00  loss: 6.480145 (6.743094)  acc1: 31.250000 (38.000000)  acc5: 62.500000 (66.000000)  time: 0.324902  data:

## Black Box on Multihead model